# Get tables and summarize situation for each council

In [ ]:
import pandas as pd
import regex as re
import json
import tabula
import pickle

    
def load_statements(filename):                  
    with open(filename) as f:
        statements = json.loads(f.read())
    return statements

def save_statements(statements, filename):
    filename = filename
    with open(filename, 'w') as f:
        f.write(json.dumps(statements))

        
        

def getTables(statements, council_name, lattice = False, stream = False, guess = True):
    
    print("Getting tables for {}".format(council_name))
    
    url = statements[council_name]
    print(url)
    
    if url == []:
        print('Statements not available (no URL provided)')
        return -1
    
    try:
    
        tables = tabula.read_pdf(url, pages = 'all', multiple_tables = True, lattice = lattice, stream = stream, guess = guess)
    except:
        print('URL not accessible\n')
        return -2
    
    if len(tables) == 0:
        print('No tables in the document')
        return -3
    
    
    newTables = list()

    for i in range(len(tables)):

        terms1 = tables[i][tables[i].apply(lambda row: \
                                          row.astype(str).str.contains('Current Assets' \
                                                                       and 'Current Liabilities', \
                                                                       flags=re.IGNORECASE,
                                                                       regex = True
                                                                      ).any(), axis=1)]

        terms2 = tables[i][tables[i].apply(lambda row: \
                                          row.astype(str).str.contains('Current Assets' \
                                                                       and 'Current Liabilities', \
                                                                       flags=re.IGNORECASE,
                                                                       regex = True
                                                                      ).any(), axis=1)]
        
        if (terms1.shape[0] != 0) or (terms2.shape[0] != 0):
            newTables.append(tables[i])
    
    
    
    if len(newTables)== 0:
        print('No tables of interest in the document')
        return -3
    
    print("\n\tFound {} tables, {} of which might be of interest\n\n".format(len(tables), len(newTables)))
    
    return newTables

def getAllTables(statements):
    
    allCouncils = {el:[] for el in list(statements)}
    
    for el in allCouncils:
        
        allCouncils[el] = getTables(statements, el)
        
        
    return allCouncils
        
    

In [ ]:
# Load statements
st_18_19 = load_statements('st_18_19')
st_19_20 = load_statements('st_19_20')

# Get rid of 'ua' in council names.
for key in list(st_18_19):
    
    newKey = key.replace(' ua', '')
    
    st_18_19[newKey] = st_18_19.pop(key)
    
for key in list(st_19_20):
    
    newKey = key.replace(' ua', '')
    
    st_19_20[newKey] = st_19_20.pop(key)


In [ ]:
# To get tables for a single council
# Use name 
# getTables(st_18_19, 'bristol ua')

# Or index
idx = 42
testBefore = getTables(st_18_19, list(st_18_19)[idx])
# Wrapping a dictionary in list() returns all keys as list
# which can be easily indexed.


In [ ]:
# # Get the tables for all the councils (18/19).
# # This returns a dict where key is the name of each council 
# # and val is a list, or a list of lists containing the 
# # relevant tables found for that council.
allTables_18_19 = getAllTables(st_18_19)

# # Save resulting dict into a file (pickle)
f = open("allTables_18_19.pkl","wb")
pickle.dump(allTables_18_19,f)
f.close()

# # To load tables (if the above has been executed beforehand)
# with open('allTables.pkl', 'rb') as handle:
#     allTablesLoaded = pickle.load(handle)


In [ ]:
# Get the tables for all the councils (19/20).
# This returns a dict where key is the name of each council 
# and val is a list, or a list of lists containing the 
# relevant tables found for that council.
allTables_19_20 = getAllTables(st_19_20)

# Save resulting dict into a file (pickle)
f = open("allTables_19_20.pkl","wb")
pickle.dump(allTables_19_20,f)
f.close()

# # To load tables
# with open('allTables.pkl', 'rb') as handle:
#     allTablesLoaded = pickle.load(handle)


In [ ]:
def summaryAllTables(allTables, idx = False):

    urlNotAccessible = []
    tablesNotAccessible = []
    noURL = []
    tablesOK = []
    
    i = 0

    for k, v in allTables.items():

        if isinstance(v, int):
            if idx == False:
                if v == -1:
                    noURL.append(k)
                if v == -2:
                    urlNotAccessible.append(k)
                if v == -3:
                    tablesNotAccessible.append(k)
                    
                
                    
            else:
                if v == -1:
                    noURL.append(i)
                if v == -2:
                    urlNotAccessible.append(i)
                if v == -3:
                    tablesNotAccessible.append(i)
                    
        else:
            tablesOK.append(k)
                    
                    
                
            i += 1    


    return noURL, urlNotAccessible, tablesNotAccessible, tablesOK            
    
    

In [ ]:
with open('allTables_18_19.pkl', 'rb') as handle:
    allTables_18_19 = pickle.load(handle)

with open('allTables_19_20.pkl', 'rb') as handle:
    allTables_18_19 = pickle.load(handle)

noURL_18_19, urlNotAccessible_18_19, tablesNotAccessible_18_19, tablesOK_18_19 = summaryAllTables(allTables_18_19)
noURL_19_20, urlNotAccessible_19_20, tablesNotAccessible_19_20, tablesOK_19_20 = summaryAllTables(allTables_19_20)


## Codes:

-1: No URL provided.
-2: URL not accessible
-3: Tables not accessible

In [ ]:
for i in range(len(urlNotAccessible_18_19)):
    print(urlNotAccessible_18_19[i], st_18_19[urlNotAccessible_18_19[i]])

In [ ]:
for i in range(len(tablesNotAccessible_18_19)):
    print(tablesNotAccessible_18_19[i], st_18_19[tablesNotAccessible_18_19[i]])

# 2018/2019

### Statements not available

1. Erewash
2. Lee valley regional park authority

### Statements not accessible (through tabula)

- central bedfordshire ua https://centralbedfordshire.app.box.com/s/jhlml56bh2wlw8xvt3npxkygoggb36i2

- stockton-on-tees ua https://www.stockton.gov.uk/media/1958618/draft-statement-of-accounts-print-version.pdf

- carlisle https://www.carlisle.gov.uk/Portals/25/Documents/Financial_Publications/2018.19%20-Final%20Statement%20of%20Accounts.pdf?timestamp=1617802691746

- high peak https://www.highpeak.gov.uk/media/4375/Draft-statement-of-accounts-2018-19/pdf/HPBC_Statement_of_Accounts_18192.pdf

- torbay ua https://www.torbay.gov.uk/media/13410/soa-1819.docx

- devon https://devoncc.sharepoint.com/:b:/s/PublicDocs/Corporate/EXixdDRZjJxGqTVjn7DTy3kB_E18vRvMlqA3fd2QMmwi2g

- torridge https://www.torridge.gov.uk/CHttpHandler.ashx?id=19566&p=0

- braintree https://www.braintree.gov.uk/downloads/file/2253/statement-of-accounts-18-19-final

- tewkesbury https://tewkesburyborough-my.sharepoint.com/personal/website_tewkesburyborough_onmicrosoft_com/_layouts/15/onedrive.aspx?originalPath=aHR0cHM6Ly90ZXdrZXNidXJ5Ym9yb3VnaC1teS5zaGFyZXBvaW50LmNvbS86ZjovZy9wZXJzb25hbC93ZWJzaXRlX3Rld2tlc2J1cnlib3JvdWdoX29ubWljcm9zb2Z0X2NvbS9FajI3cnc2d19jeERyN1drcVI0aVRITUJZUEhqX0FBRkpqcktwZzcwazlMQTRnP3J0aW1lPUMtWlBtdFQ1MkVn&id=%2Fpersonal%2Fwebsite%5Ftewkesburyborough%5Fonmicrosoft%5Fcom%2FDocuments%2FTewkesbury%20Borough%20Council%20%28TBC%29%2FAbout%20the%20Council%2FBudgets%20and%20finance%2FStatement%20of%20accounts%2F2018%2D2019%20%2D%20Statement%20of%20accounts%2FFinancial%20statements%2C%202018%2D19%2Epdf&parent=%2Fpersonal%2Fwebsite%5Ftewkesburyborough%5Fonmicrosoft%5Fcom%2FDocuments%2FTewkesbury%20Borough%20Council%20%28TBC%29%2FAbout%20the%20Council%2FBudgets%20and%20finance%2FStatement%20of%20accounts%2F2018%2D2019%20%2D%20Statement%20of%20accounts

- fareham http://www.fareham.gov.uk/PDF/about_the_council/draft-statoa1819.pdf

- wyre forest https://wyreforestdc.gov.uk/media/4612869/2018-19-Statement-of-Accounts.pdf

- broxbourne https://www.broxbourne.gov.uk/downloads/file/692/2018-19-statement-of-accounts-final-with-signatures

- dartford https://www.dartford.gov.uk/__data/assets/pdf_file/0005/833153/Statement-of-Accounts-2018-19-with-audit-adjustments-30-09.pdf

- gravesham https://www.gravesham.gov.uk/__data/assets/pdf_file/0004/813055/Statement-of-Accounts-2018-19-Post-Audit-for-Web.pdf

- tonbridge & malling https://www.tmbc.gov.uk/__data/assets/pdf_file/0010/783946/Statement-of-Accounts-1819.pdf
 
- burnley https://www.burnley.gov.uk/sites/default/files/Notice%20of%20Conclusion%20of%20Audit%202018-19.pdf

- lincolnshire https://www.lincolnshire.gov.uk/downloads/file/1952/statement-of-accounts-2018-19

- broadland https://www.broadland.gov.uk/download/downloads/id/5433/statement_of_accounts_2018_to_2019.pdf

- york ua https://www.york.gov.uk/downloads/file/2236/cyc-statement-of-accounts-2018-to-2019

- hambleton http://hambleton.gov.uk/download/downloads/id/4756/statement_of_accounts_2018-19.pdf

- harrogate https://www.harrogate.gov.uk/downloads/file/1336/statement-of-accounts-2018-2019

- northamptonshire https://www.northamptonshire.gov.uk/councilservices/council-and-democracy/budgets-and-spending/Documents/Draft%20NCC%20Statement%20of%20Accounts%202018-19.pdf

- northampton https://www.northampton.gov.uk/download/downloads/id/11571/annual-governance-statement-201819.pdf

- lichfield https://www.lichfielddc.gov.uk/downloads/file/1251/statement-of-accounts-2018-19

- staffordshire moorlands https://www.staffsmoorlands.gov.uk/media/4471/SMDC-Statement-of-Accounts-2018-19/pdf/SMDC_Statement_of_Accounts_18_-_19.pdf?m=1599043474207

- mid suffolk https://www.babergh.gov.uk/assets/Finance/Mid-Suffolk-DC-Statement-of-Accounts-2018-19-published.pdf

- reigate & banstead https://www.reigate-banstead.gov.uk/download/downloads/id/5526/unaudited_annual_accounts_2018-19.pdf

- manchester https://secure.manchester.gov.uk/download/downloads/id/27025/audited_accounts_2018-19.pdf

- sefton https://www.sefton.gov.uk/media/3281/statement-of-accounts-2018-2019-post-ag.pdf

- barnsley https://www.barnsley.gov.uk/media/17922/bmbc-audited-statement-of-accounts-201819.pdf

- rotherham https://www.rotherham.gov.uk/downloads/file/1907/statement-of-accounts-2018-19

- bradford https://www.bradford.gov.uk/media/5531/audited-statement-of-accounts-2018-19.pdf

- calderdale https://www.calderdale.gov.uk/v2/sites/default/files/Statement-of-Accounts-18-19.pdf

- leeds https://www.leeds.gov.uk/docs/Final%20Accounts%202018-19.pdf

- city of london https://www.cityoflondon.gov.uk/assets/About-us/statement-of-accounts-2018-19.pdf

- hackney https://drive.google.com/file/d/1tQAZ5KwWE2LwRHcW6zJe3IVb0crE4Wmu/view?usp=sharing

- southwark https://www.southwark.gov.uk/assets/attach/10125/SOA-2018-19-Skeleton-JC-MASTER-31.07.19-22.00hrs-.pdf

- westminster https://www.westminster.gov.uk/media/document/annual-accounts-2018-19

- ealing https://www.ealing.gov.uk/download/downloads/id/15651/statement_of_accounts_2018-19_audited.pdf

- harrow https://www.harrow.gov.uk/downloads/file/26425/audited-statement-of-accounts-2018-19

- newham https://www.newham.gov.uk/downloads/file/476/statementofaccounts201819

- greater london authority https://www.london.gov.uk/sites/default/files/gla_group_audited_2018.19_statement_of_accounts-audit_signature.pdf

- greater manchester combined authority https://www.greatermanchester-ca.gov.uk/media/2481/gmca-statement-of-accounts-2018-19-with-audit-opinion.pdf

- sheffield city region combined authority https://www.sheffield.gov.uk/content/dam/sheffield/docs/your-city-council/archived-statement-of-accounts/Sheffield%20City%20Council%20Audited%20Statement%20of%20Accounts%202018-19%201.pdf

- tees valley combined authority https://teesvalley-ca.gov.uk/wp-content/uploads/2019/12/Annual-Statement-of-Accounts-18-19.pdf

- north east combined authority https://northeastca.gov.uk/wp-content/uploads/2019/07/NECA-Statement-of-Accounts-2018-19.pdf

- northumberland national park authority https://www.northumberlandnationalpark.org.uk/wp-content/uploads/2019/07/ITEM-3-Appendix-1-Statement-of-Accounts-1819-amended.pdf

- south downs national park authority https://www.southdowns.gov.uk/wp-content/uploads/2019/05/Unaudited-Statement-of-Accounts-2018-19.pdf


### Table not accessible within statement

- aylesbury vale https://www.aylesburyvaledc.gov.uk/sites/default/files/page_downloads/Statement%20of%20Accounts%202018-19%20Published.pdf

- huntingdonshire https://www.huntingdonshire.gov.uk/media/4080/statement-of-accounts-2018-19.pdf

- redcar & cleveland ua https://www.redcar-cleveland.gov.uk/about-the-council/budgets/statement-of-accounts/Documents/Summary%20Accounts%202018-19.pdf

- barrow-in-furness https://www.barrowbc.gov.uk/_resources/assets/attachment/full/0/8017.pdf

- south lakeland https://www.southlakeland.gov.uk/media/5063/approved-budget-book-2018-9.pdf

- bolsover https://www.bolsover.gov.uk/component/edocman/12173-statement-of-accounts-2018-19/download

- north devon https://www.northdevon.gov.uk/media/379438/soa-2018-19.pdf

- teignbridge https://www.teignbridge.gov.uk/media/8537/2018-19-final-statement-of-accounts-with-strata-adjustments.pdf

- hastings https://www.hastings.gov.uk/content/my_council/budgets_finance/budgets_finance/Hastings_Borough_Council_Budget_2018-19.pdf

- rother https://www.rother.gov.uk/wp-content/uploads/2020/01/Audited_statement_of_accounts_-_final.pdf

- brentwood https://www.brentwood.gov.uk/pdf/30012020163222000000.pdf

- chelmsford https://www.chelmsford.gov.uk/easysiteweb/getresource.axd?assetid=2431277&type=0&servicetype=1

- rochford https://www.rochford.gov.uk/sites/default/files/finance_annual_report_june_2019_0.pdf

- portsmouth ua https://www.portsmouth.gov.uk/wp-content/uploads/2020/05/Portsmouth-City-Council-audited-statement-of-accounts-2018-19.pdf

- east hampshire https://www.easthants.gov.uk/statement-accounts-2018-19-pdf-87-mb

- rushmoor https://www.rushmoor.gov.uk/CHttpHandler.ashx?id=21352&p=0

- winchester https://www.winchester.gov.uk/assets/attach/19942/Annual-Financial-Report-1819-V6.4.pdf

- worcestershire http://www.worcestershire.gov.uk/download/downloads/id/11356/financial_accounts_20182019.pdf

- bromsgrove https://www.bromsgrove.gov.uk/media/4618533/Bromsgrove-Statement-of-Accounts-2018-19-Final.pdf

- worcester https://www.worcester.gov.uk/component/fileman/file/PDF%20Documents/About%20your%20council/Corporate_Reports/2018-2019/Statement%20of%20Accounts%20and%20Audit%20Opinion%202018-2019.pdf?routed=1&container=fileman-files

- hertfordshire https://www.north-herts.gov.uk/sites/northherts-cms/files/Signed%20and%20Audited%20Statement%20of%20Accounts%202018-19.pdf

- east hertfordshire https://eastherts.fra1.digitaloceanspaces.com/s3fs-public/2020-02/SOA%202018-19%20tables%20V6%20FINAL%20For%20Publication%20with%20Signatures_AM%20%282%29.pdf

- north hertfordshire https://www.north-herts.gov.uk/sites/northherts-cms/files/Signed%20and%20Audited%20Statement%20of%20Accounts%202018-19.pdf

- st albans https://www.stalbans.gov.uk/sites/default/files/documents/publications/council/SoA%202018-19%20%282019.07.31%29%20Audited%20Final_tcm15-67768.pdf

- tunbridge wells http://tunbridgewells.gov.uk/__data/assets/pdf_file/0009/338436/TWBC-Annual-Financial-Report-2018-19.pdf

- blackpool ua https://www.blackpool.gov.uk/Your-Council/Transparency-and-open-data/Documents/Audited-statement-of-accounts-2018-19.pdf

- lancaster https://www.lancaster.gov.uk/assets/attach/5962/Lancaster%20City%20Council%20SoA%2018-19%20Signed.pdf

- ribble valley https://www.ribblevalley.gov.uk/download/downloads/id/12370/statement_of_accounts_20182019.pdf

- harborough http://www.harborough.gov.uk/download/downloads/id/5891/statement_of_accounts_201819_with_audit_opinion.pdf

- oadby & wigston https://www.oadby-wigston.gov.uk/files/documents/unaudited_satement_of_accounts_2018_19/Audited%20Statement%20of%20Accounts%202018-19.pdf

- south kesteven http://moderngov.southkesteven.gov.uk/documents/s23531/Statement%20of%20Accounts%20GAC%20Final%20201819.pdf

- north norfolk https://www.north-norfolk.gov.uk/media/5097/nndc-statement-of-accounts-2018-19-subject-to-audit.pdf

- east northamptonshire https://www.east-northamptonshire.gov.uk/download/downloads/id/11301/draft_statement_of_accounts_2018-19_%E2%80%93_unaudited_republished_25th_july_2019.pdf

- ashfield https://www.ashfield.gov.uk/media/8d854d5245a5465/ashfield-dc-acr-v5.pdf

- gedling https://www.gedling.gov.uk/media/gedlingboroughcouncil/documents/datasets/Audited%20Statement%20of%20Accounts%201819%20v3-Final.pdf

- oxfordshire https://www.oxfordshire.gov.uk/sites/default/files/file/council-tax-and-finance/AnnualAuditLetter2018-19.pdf

- somerset west and taunton https://www.somersetwestandtaunton.gov.uk/media/2132/tdbc-audited-statement-of-accounts-2018-19.pdf

- cannock chase https://www.cannockchasedc.gov.uk/sites/default/files/cdc_statement_of_accounts_18-19_full_statement_-_subject_to_audit.pdf

- east staffordshire https://www.eaststaffsbc.gov.uk/sites/default/files/docs/finance/Statement-of-Accounts-2018-19.pdf

- elmbridge https://www.elmbridge.gov.uk/_resources/assets/attachment/full/0/7142.pdf

- tandridge https://www.tandridge.gov.uk/Portals/0/Documents/Your-council/Freedom-of-info-and-data/Finance/Financial%20statements/Audited-Statement-of-Accounts-2018-19.pdf

- north warwickshire https://www.northwarks.gov.uk/download/downloads/id/8590/financial_statements_2018-2019_%E2%80%93_audited.pdf

- nuneaton & bedworth https://www.nuneatonandbedworth.gov.uk/download/downloads/id/3250/unaudited_statement_of_accounts_2018-2019.pdf

- warwick https://www.warwickdc.gov.uk/download/downloads/id/5660/audited_201819_financial_statements.pdf

- west sussex https://www.westsussex.gov.uk/media/13011/annual_report_2018_19.pdf

- arun https://www.arun.gov.uk/download.cfm?doc=docm93jijm4n11952.pdf&ver=11952

- crawley https://crawley.gov.uk/sites/default/files/documents/PUB352811.pdf

- isles of scilly https://www.scilly.gov.uk/sites/default/files/document/council/CIOS%20Statement%20of%20Accounts%202018-19%20Final%20to%20Full%20Council%20with%20opinion_0.pdf

- north tyneside https://my.northtyneside.gov.uk/sites/default/files/web-page-related-files/2018-19%20Statement%20of%20Accounts%20-%20FINAL%20with%20Audit%20Amendments%2001-08-2019.pdf

- coventry https://edemocracy.coventry.gov.uk/documents/s44283/Unaudited%20201819%20Statement%20of%20Accounts%20-%20Appendix.pdf

- dudley https://www.dudley.gov.uk/media/12620/statement-of-accounts-2018-19.pdf

- brent https://www.brent.gov.uk/media/16413900/2018-19-accounts.pdf

- enfield https://new.enfield.gov.uk/services/your-council/statement-of-accounts/final-audited-statement-of-accounts-2018-19.pdf

Probably won't be using these:

- western riverside waste authority https://wrwa.gov.uk/wp-content/uploads/2020/03/Signed-financial-statements-WRWA-31-March-2019.pdf

- north of tyne combined authority https://www.northoftyne-ca.gov.uk/wp-content/uploads/2020/08/NTCAAnnualFinancialReport2018-19.pdf

- dartmoor national park authority https://www.dartmoor.gov.uk/__data/assets/pdf_file/0032/96269/2018-19-Audited-Statement-of-Accounts-and-Auditor-Opinion-for-publication.pdf

- lake district national park authority https://www.lakedistrict.gov.uk/__data/assets/pdf_file/0018/143343/LDNPA_1819_Q1.pdf

- peak district national park authority https://www.peakdistrict.gov.uk/__data/assets/pdf_file/0023/84047/Budget-Report-2018-2019.pdf

- yorkshire dales national park authority https://www.yorkshiredales.org.uk/wp-content/uploads/sites/13/2019/06/Budget-2018-19-1.pdf

In [ ]:
for i in range(len(urlNotAccessible_19_20)):
    
    print(urlNotAccessible_19_20[i], st_19_20[urlNotAccessible_19_20[i]])

In [ ]:
for i in range(len(tablesNotAccessible_19_20)):
    
    print(tablesNotAccessible_19_20[i], st_19_20[tablesNotAccessible_19_20[i]])

# 2018/2019

### Statements not available

- telford and the wrekin
- wellingborough
- east northamptonshire
- blackpool
- copeland
- south cambridgeshire 
- reading
- lewes
- sedgemoor
- corby
- copeland 
- isles of scilly 
- greenwich 

### Statements not accessible (through tabula)

- central bedfordshire https://centralbedfordshire.app.box.com/s/30728jvcrod2uxyxggvy79m3ybxlifbo
- stockton-on-tees https://www.stockton.gov.uk/media/1961934/final-accounts-november-2020-signed-version.pdf
- carlisle https://www.carlisle.gov.uk/Portals/24/Documents/Local_Plan/Local%20Development%20Scheme/LDS%20Update%202019-2022.pdf?timestamp=1562752549040
- high peak https://www.highpeak.gov.uk/media/6237/High-Peak-Annual-Report-2019-20/pdf/High_Peak_Annual_Report_2019-20.pdf?m=1606904616313
- teignbridge https://www.teignbridge.gov.uk/media/9267/2019-20-final-statement-of-accounts-2019-2020-with-strata-adjustments-accessable-version-v3-signed-copy.docx
- darlington https://www.darlington.gov.uk/media/11981/dbc_2019-20_soa_unaudited.pdf
- braintree https://www.braintree.gov.uk/downloads/file/2911/statement-of-accounts-19-20
- tewkesbury https://tewkesburyborough-my.sharepoint.com/personal/website_tewkesburyborough_onmicrosoft_com/_layouts/15/onedrive.aspx?originalPath=aHR0cHM6Ly90ZXdrZXNidXJ5Ym9yb3VnaC1teS5zaGFyZXBvaW50LmNvbS86ZjovZy9wZXJzb25hbC93ZWJzaXRlX3Rld2tlc2J1cnlib3JvdWdoX29ubWljcm9zb2Z0X2NvbS9FdWlFNDJPNFNSZEJ2RE9BTm5YbUFESUJYMVpDajVneVZOMkctV0Z3V1lkRGF3P3J0aW1lPUl6V3ZPQ2pfMkVn&id=%2Fpersonal%2Fwebsite%5Ftewkesburyborough%5Fonmicrosoft%5Fcom%2FDocuments%2FTewkesbury%20Borough%20Council%20%28TBC%29%2FAbout%20the%20Council%2FBudgets%20and%20finance%2FStatement%20of%20accounts%2F2019%2D2020%20%2D%20Statement%20of%20accounts%2FFinancial%20statements%2C%202019%2D20%2Epdf&parent=%2Fpersonal%2Fwebsite%5Ftewkesburyborough%5Fonmicrosoft%5Fcom%2FDocuments%2FTewkesbury%20Borough%20Council%20%28TBC%29%2FAbout%20the%20Council%2FBudgets%20and%20finance%2FStatement%20of%20accounts%2F2019%2D2020%20%2D%20Statement%20of%20accounts
- fareham http://www.fareham.gov.uk/PDF/about_the_council/statoa1920.pdf
- broxbourne https://www.broxbourne.gov.uk/downloads/file/916/statement-of-accounts-2019-20
- canterbury https://www.canterbury.gov.uk/download/downloads/id/1382/statement_of_accounts_2019-2020.pdf
- blackburn with darwen https://www.blackburn.gov.uk/sites/default/files/media/pdfs/Statement%20of%20Accounts%202019-20%20DRAFT-ua.pdf
- burnley https://www.burnley.gov.uk/sites/default/files/Burnley%20BC%202019-20%20Statement%20of%20Accounts%20DRAFT.pdf
- hyndburn https://www.hyndburnbc.gov.uk/download-package/statement-of-accounts/?wpdmdl=6004&ind=DXXGJBo1H9rtIw5RzQCXt91EGVNV0J-vO_VlAZFC5k2CWQf8wXja5S_O75V62B0I97GPE1kN3jpL9aiEVcLs8Obep9or6iVRTROm8cL73B3IWgUS4kvDyeSzyLyt4fB1Q6cOjlUWLNZFvTYjdB0-bA
- rossendale https://www.rossendale.gov.uk/download/downloads/id/16245/statement_of_accounts_2019-20.pdf
- hinckley & bosworth https://www.hinckley-bosworth.gov.uk/download/downloads/id/7036/unaudited_statement_of_accounts_2019-2020.pdf
- lincolnshire https://www.lincolnshire.gov.uk/downloads/file/4342/lcc-draft-statement-of-accounts-2019-20
- broadland https://www.broadland.gov.uk/download/downloads/id/6319/statement_of_accounts_2019-20.pdf
- york https://www.york.gov.uk/downloads/file/5810/draft-statement-of-accounts-2019-to-2020
- hambleton https://www.hambleton.gov.uk/downloads/file/104/statement-of-accounts-2019-20
- harrogate https://www.harrogate.gov.uk/downloads/file/4016/statement-of-accounts-2019-2020
- oxford https://www.oxford.gov.uk/download/downloads/id/7403/statement_of_accounts_2019-20.pdf
- somerset https://somersetcc.sharepoint.com/sites/SCCPublic/Council%20Finances/Forms/AllItems.aspx?id=%2Fsites%2FSCCPublic%2FCouncil%20Finances%2FSCC%20Statement%20of%20Accounts%20201920%2Epdf&parent=%2Fsites%2FSCCPublic%2FCouncil%20Finances&p=true&originalPath=aHR0cHM6Ly9zb21lcnNldGNjLnNoYXJlcG9pbnQuY29tLzpiOi9zL1NDQ1B1YmxpYy9FZXNCdUpuX0k0MUdqQ2NYWUdwTHJqWUI3VlBkZW05TmpGanNLUFFCVlI0bURRP3J0aW1lPVV3ajY0UjNfMkVn
- staffordshire moorlands https://www.staffsmoorlands.gov.uk/media/5007/Statement-of-Accounts-2019---2020/pdf/01_Staffordshire_Moorlands_PLEASE_USE_2.pdf?m=1606749725290
- reigate & banstead http://www.reigate-banstead.gov.uk/download/downloads/id/6443/unaudited_statement_of_accounts_2019-20.pdf
- chichester https://www.chichester.gov.uk/media/34403/Statement-of-Accounts-2019-20/doc/Statement_of_Accounts_2019-20_(3).docx

- sefton https://www.sefton.gov.uk/media/3282/statement-of-accounts-2019-20-published-draft.pdf
- barnsley https://www.barnsley.gov.uk/media/16387/barnsley-council-draft-annual-governance-statement.pdf
- rotherham https://www.rotherham.gov.uk/downloads/file/1992/statement-of-accounts-2019-20
- sheffield https://sheffieldcityregion.org.uk/wp-content/uploads/2020/11/20-SCRMCA-Final-Signed-Electronic-Signatures-Removed-26-Nov-2020-unprotected.pdf
- birmingham https://www.birmingham.gov.uk/download/downloads/id/17119/birmingham_city_council_draft_statement_of_accounts_for_the_financial_year_2019_to_2020.pdf
- coventry https://www.coventry.gov.uk/download/downloads/id/33309/statement_of_accounts_2019-2020_draft.pdf
- bradford https://www.bradford.gov.uk/media/6131/audited-statement-of-accounts-2019-20.pdf
- calderdale https://www.calderdale.gov.uk/v2/sites/default/files/statacc19-20-post-audit.pdf
- leeds https://www.leeds.gov.uk/docs/Draft%20Statement%20of%20Accounts%202019-20.pdf
- city of london https://www.cityoflondon.gov.uk/assets/About-us/audited-city-fund-accounts-2019-2020.pdf
- hackney https://drive.google.com/file/d/120s8D-aQImsu6CFPhxNw24m8UNfWUZT0/view
- southwark https://www.southwark.gov.uk/assets/attach/28611/SOA-2019-20-Skeleton-JC-MASTER-30.11.20-20.00hrs-.pdf
- ealing https://www.ealing.gov.uk/download/downloads/id/15999/statement_of_accounts_audited_2019-20.pdf
- harrow https://www2.harrow.gov.uk/documents/s166475/2.0%20P%20Board%20170920%20Annual%20Report%20cover.pdf
- kingston upon thames https://www.kingston.gov.uk/downloads/file/53/rbk-final-statement-of-accounts-2019-20
- newham https://www.newham.gov.uk/downloads/file/2602/statement-of-accounts-2019-20
- greater london authority https://www.london.gov.uk/sites/default/files/gla_group_2019-20_statement_of_accounts_unaudited.pdf
- merseyside waste disposal authority https://www.sefton.gov.uk/media/3282/statement-of-accounts-2019-20-published-draft.pdf
- sheffield city region combined authority https://sheffieldcityregion.org.uk/wp-content/uploads/2020/11/20-SCRMCA-Final-Signed-Electronic-Signatures-Removed-26-Nov-2020-unprotected.pdf
- the west yorkshire combined authority https://www.leeds.gov.uk/docs/Draft%20Statement%20of%20Accounts%202019-20.pdf
- tees valley combined authority https://teesvalley-ca.gov.uk/wp-content/uploads/2020/01/Annual-Report-2019-digital.pdf
- south downs national park authority https://www.southdowns.gov.uk/wp-content/uploads/2020/07/SDNPA_AnnualReview_2019-2020-06b.pdf


### Table not accessible within statement

- bracknell forest https://www.bracknell-forest.gov.uk/sites/default/files/documents/draft-financial-statements-2019-to-2020.pdf
- slough http://democracy.slough.gov.uk/documents/s63063/Going%20Concern%20Statement%2018-19%20Accounts.pdf
- wokingham https://www.wokingham.gov.uk/_resources/assets/attachment/full/0/530578.pdf
- chiltern https://www.chiltern.gov.uk/media/15895/19-20-Notice-of-Conclusion-of-Audit/pdf/Conclusion_notice_Chiltern_District_Council.pdf?m=637424377035500000
- wycombe https://www.wycombe.gov.uk/uploads/public/documents/About-the-council/Council-budget-and-spending/2020-11-30-WDC-STATEMENT-OF-ACCOUNTS-19-20-Post-Audit-Committee-Adjustme....pdf
- halton https://www3.halton.gov.uk/Pages/councildemocracy/pdfs/Finance/statementofaccounts/Accounts2019-20/statement.pdf
- north devon https://www.northdevon.gov.uk/media/380366/soa-2019-20-final-audit-adjustments-nov-2020.pdf
- wealden https://www.wealden.gov.uk/UploadedFiles/Final-Version-WDC-accounts-2019-20.pdf
- thurrock https://www.thurrock.gov.uk/sites/default/files/assets/documents/acco_2020_annual_accounts_v02.pdf
- colchester https://cbccrmdata.blob.core.windows.net/noteattachment/CBC-Our%20Council-Governance-Statement%20of%20Accounts%202019:20.pdf
- forest of dean https://www.fdean.gov.uk/media/8d8954f3ad2c9c6/statement-of-accounts-2019-to-2020.pdf
- eastleigh https://www.eastleigh.gov.uk/media/7663/2019-20-statement-of-accounts-second-draft-version.pdf
- rushmoor https://www.rushmoor.gov.uk/CHttpHandler.ashx?id=21199&p=0
- winchester https://www.winchester.gov.uk/assets/attach/25874/Unaudited-Annual-Financial-Report-201920.pdf
- worcestershire https://www.worcestershire.gov.uk/download/downloads/id/12714/financial_accounts_20192020.pdf
- bromsgrove https://www.bromsgrove.gov.uk/media/5762998/Bromsgrove-District-Council-Draft-Statement-of-Accounts-2019-20.pdf
- redditch https://www.redditchbc.gov.uk/media/5654436/Redditch-Borough-Council-Statement-of-Accounts-2019-20-DRAFT.PDF
- worcester https://www.worcester.gov.uk/?option=com_fileman&view=file&routed=1&name=Draft%20Statement%20of%20Accounts%202019-20%20unaudited.pdf&folder=PDF%20Documents%2FAbout%20your%20council%2FCorporate_Reports%2F2020-2021&container=fileman-files
- north hertfordshire https://www.north-herts.gov.uk/sites/northherts-cms/files/Unaudited%20Statement%20of%20Accounts%202019-20_0.pdf
- three rivers https://www.threerivers.gov.uk/download?id=48466
- watford https://www.watford.gov.uk/download/downloads/id/3393/draft_statement_of_accounts_watford_borough_council_2019-20.pdf
- the medway towns https://www.medway.gov.uk/download/downloads/id/5150/draft_statement_of_accounts_2019_to_2020.pdf
- kent https://www.kent.gov.uk/__data/assets/pdf_file/0019/115804/Statement-of-accounts-2019-20.pdf
- ribble valley https://www.ribblevalley.gov.uk/download/downloads/id/12995/statement_of_accounts_20192020.pdf
- melton http://www.melton.gov.uk/download/downloads/id/5320/statement_of_accounts_2019-2020_audit_not_complete_by_301120.pdf
- north west leicestershire https://www.nwleics.gov.uk/files/documents/statement_of_accounts_2019_20/Statement+of+Accounts+Final.pdf
- north norfolk https://www.north-norfolk.gov.uk/media/6036/nndc-soa-2019-20-draft.pdf
- northampton http://pensions.northamptonshire.gov.uk/app/uploads/2020/12/AnnualReportAndStatementOfAccounts2019-2020unaudited.pdf
- northumberland https://www.northumberland.gov.uk/NorthumberlandCountyCouncil/media/About-the-Council/Finance/Public-Inspection-Notice-2019-2020.pdf
- gedling https://democracy.gedling.gov.uk/documents/s25738/Appx%202%20-%20AUDITED%20Statement%20of%20Accounts%202019-20%2019.11.20.pdf
- newark & sherwood https://www.newark-sherwooddc.gov.uk/media/newarkandsherwood/imagesandfiles/finance/statementofaccounts/StatementofAccounts2019-2020.pdf
- somerset west and taunton https://www.somersetwestandtaunton.gov.uk/media/2600/swt-audited-statement-of-accounts-2019-20.pdf
- cannock chase https://www.cannockchasedc.gov.uk/sites/default/files/statement_of_accounts_19-20_new.pdf
- babergh https://www.midsuffolk.gov.uk/assets/Strategic-Planning/AMR/5HLS-2020/Babergh-5YHLS-Report-Consultation-Issue-Aug-2020.pdf
- west suffolk https://www.westsuffolk.gov.uk/Council/Finance_and_Statistics/upload/WS-2019-20-Audited-Statement-of-Accounts-internet.pdf
- surrey heath https://www.surreyheath.gov.uk/sites/default/files/documents/council/finance/DRAFT%20Statement%20of%20Accounts%201920%20v3%20final%20post%20audit%20copy.pdf
- tandridge https://www.tandridge.gov.uk/Portals/0/Documents/Your-council/Freedom-of-info-and-data/Finance/Financial%20statements/Draft%20Statement%20of%20Accounts%20for%20Publication%20(002).pdf?ver=2020-09-09-155713-823
- north warwickshire https://www.northwarks.gov.uk/download/downloads/id/8535/financial_statements_2019-2020_%E2%80%93_unaudited.pdf
- nuneaton & bedworth https://www.nuneatonandbedworth.gov.uk/download/downloads/id/4219/audited_statement_of_accounts_2019-2020.pdf
- warwick https://www.warwickdc.gov.uk/download/downloads/id/6078/201920_draft_unaudited_statement_of_accounts.pdf
- arun https://www.arun.gov.uk/download.cfm?doc=docm93jijm4n16178.pdf&ver=16701
- crawley https://crawley.gov.uk/sites/default/files/2020-06/Statement%20of%20Accounts%202019-20.pdf
- swindon https://www.wiltshire.gov.uk/media/4952/Statement-of-Accounts-2019-2020/pdf/Published_SOA_28_August.pdf?m=637393250075030000
- wiltshire https://www.wiltshire.gov.uk/media/4952/Statement-of-Accounts-2019-2020/pdf/Published_SOA_28_August.pdf?m=637393250075030000
- north tyneside https://my.northtyneside.gov.uk/sites/default/files/web-page-related-files/Audited%20Statement%20of%20Accounts%202019-20.pdf
- kensington & chelsea https://www.rbkc.gov.uk/sites/default/files/atoms/files/Statement%20of%20Accounts%202019-20%20unaudited.docx.pdf
- westminster https://committees.westminster.gov.uk/documents/s40636/COWAAL19-20%20DRAFT.pdf
- redbridge https://www.redbridge.gov.uk/media/8326/draft-unaudited-statement-of-accounts-2019-20-on-redbridge-website.pdf
- north london waste authority https://www.nlwa.gov.uk/sites/default/files/2020-07/08%202019-20%20Financial%20Outturn%20and%202020-21%20Update.pdf
- north east combined authority https://www.northoftyne-ca.gov.uk/wp-content/uploads/2020/09/DraftNarrativeReport12019-2020.pdf
- north of tyne combined authority https://www.northoftyne-ca.gov.uk/wp-content/uploads/2020/09/DraftNarrativeReport12019-2020.pdf
- dartmoor national park authority https://www.dartmoor.gov.uk/__data/assets/pdf_file/0020/342074/2019-20-Audited-Statement-of-Accounts.pdf
- yorkshire dales national park authority https://www.yorkshiredales.org.uk/wp-content/uploads/sites/13/2021/04/Item-12-Annual-Audit-Letter-combined.pdf

# Cleaning tables